# Development of Tank Blending System Models

In [ ]:
import casadi as cas
import numpy as np
import matplotlib.pyplot as plt

from cas_models.continuous_time.models import (
    StateSpaceModelCT,
    make_n_step_simulation_function
)
from cas_models.discrete_time.models import (
    StateSpaceModelDT, StateSpaceModelDTFromCTRK4
)

## Surge Tank with Mass Concentration

In [ ]:
# Physical constants
D = 5  # tank diameter [m]
A = np.pi * D  # tank cross-sectional area [m^2]

# System states: 
#  x[0] : Tank level, L [m]
#  x[1] : Total mass of suspended mineral in tank, m [tons]
n = 2

# Inputs
#  u[0] : volumetric flowrate into tank, v_dot_in [m^3/hr]
#  u[1] : density of fluid entering tank, rho_in [tons/m^3]
#  u[2] : volumetric flowrate out of tank, v_dot_out [m^3/hr]
nu = 3

# Outputs
#  y[0] : Tank level, L [m]
#  y[1] : Total mass of suspended mineral in tank, m [tons]
ny = 2

# Define the ODE right-hand side
x = cas.MX.sym('x', n)  
u = cas.MX.sym('u', nu)

dL_dt = (u[0] - u[2]) / A
dm_dt = u[0] * u[1] - u[2] * x[1] / (x[0] * A)

rhs = cas.vertcat(dL_dt, dm_dt)
assert rhs.shape == (n, 1)

t = cas.MX.sym('t')
f = cas.Function(
    "f",
    [t, x, u],
    [rhs],
    ["t", "x", "u"],
    ["rhs"]
)

# Define output function
y = x

h = cas.Function(
    "h",
    [t, x, u],
    [y],
    ["t", "x", "u"],
    ["y"]
)

print(f)
print(h)

In [ ]:
tank_model_ct = StateSpaceModelCT(f, h, n, nu, ny)
tank_model_ct

In [ ]:
dt = 0.25
tank_model_dt = StateSpaceModelDTFromCTRK4(tank_model_ct, dt)
tank_model_dt

In [ ]:
# Test 1 - no change in volume or concentration
L = 5
conc = 0.2
m = A * L * conc
v_dot_in = 1
v_dot_out = 1

# Function arguments
t = 0.0
dt = 0.25
xk = cas.vertcat(L, m)
uk = cas.vertcat(v_dot_in, conc, v_dot_out)
xkp1 = tank_model_dt.F(t, xk, uk)
print(xkp1)
assert np.allclose(xkp1, [[L], [m]])

# Check output function
yk = tank_model_dt.H(t, xk, uk)
print(yk)
assert np.allclose(yk, xk)

In [ ]:
# Test 2 - increasing volume
L = 5
conc = 0.2
m = A * L * conc
v_dot_in = 2
v_dot_out = 1

# Function arguments
t = 0.0
dt = 0.25
xk = cas.vertcat(L, m)
uk = cas.vertcat(v_dot_in, conc, v_dot_out)
xkp1 = tank_model_dt.F(t, xk, uk)
L2 = L + (v_dot_in - v_dot_out) * dt / A
m2 = (A * L + (v_dot_in - v_dot_out) * dt) * conc
print(xkp1)
assert np.allclose(xkp1, [[L2], [m2]])

In [ ]:
# Test 3 - increasing concentration with no out flow
L = 5
conc = 0.2
m = A * L * conc
v_dot_in = 1
conc_in = 2 * conc
v_dot_out = 0

# Function arguments
t = 0.0
dt = 0.25
xk = cas.vertcat(L, m)
uk = cas.vertcat(v_dot_in, conc, v_dot_out)
xkp1 = tank_model_dt.F(t, xk, uk)
L2 = L + (v_dot_in - v_dot_out) * dt / A
m2 = (A * L * conc + (v_dot_in * dt) * conc_in)
print(xkp1, [L2, m2])
assert np.allclose(xkp1, [[L2], [m2]], atol=0.1)  # TODO: figure out why this is not close

In [ ]:
# Test 4 - response with zero initial concentration in tank
L = 5
m = 0
v_dot_in = 1
conc_in = 0.5
v_dot_out = 1

# Function arguments
t = 0.0
dt = 0.25
xk = cas.vertcat(L, m)
uk = cas.vertcat(v_dot_in, conc_in, v_dot_out)
nT = 1000
X = np.full((nT+1, 2), np.nan)
for k in range(nT):
    X[k, :] = xk.T
    xk = tank_model_dt.F(t, xk, uk)
    t += dt
X[nT, :] = xk.T
m_final = A * L * conc_in
print(xk)
assert np.allclose(xk, [[L], [37.64189612141962]])

In [ ]:
def make_tank_tsplot(X, dt, m_final=None, y_lim=([-0.5, 45.5]), title="Model States"):
    nT = X.shape[0] - 1
    t = dt * np.arange(0, nT+1)
    plt.plot(t, X)
    if m_final is not None:
        plt.hlines(m_final, t[0], t[-1], color='C1', linestyle='--')
    plt.grid()
    plt.xlabel('Time $t$')
    plt.ylabel(r'Output Variables')
    plt.ylim(y_lim)
    plt.legend(labels=['Tank level', 'Total mass', 'Mass in steady-state'], loc='best')
    plt.title(title)
    return plt.gca()

ax = make_tank_tsplot(X, dt, m_final)
plt.show()

In [ ]:
nT = 1000
simulate = make_n_step_simulation_function(
    tank_model_dt.F, 
    tank_model_dt.H, 
    tank_model_dt.n, 
    tank_model_dt.nu, 
    tank_model_dt.ny, 
    nT,
    params=tank_model_ct.params
)
simulate

In [ ]:
# Simulation time
Ts = 0.25
t_eval = Ts * np.arange(nT+1)

# Input sequence
v_dot_in = 1
conc_in = 0.5
v_dot_out = 1
U = np.full((nT, nu), np.nan)
U[:, 0] = v_dot_in
U[:, 1] = conc_in
U[:, 2] = v_dot_out
assert U.shape == (nT, nu)

# Initial condition
L = 5
m = 0
x0 = [L, m]

assert t_eval.shape == (nT + 1, )
X, Y = simulate(t_eval, U, x0)
assert X.shape == (nT + 1, n)
assert Y.shape == (nT + 1, ny)

assert np.allclose(X[-1, :], [L, 37.64189612141962])

In [ ]:
ax = make_tank_tsplot(X, dt, m_final)
plt.show()

In [ ]:
# Test 5 - response with initial mass and zero inlet concentration in tank
L = 5
m = 39.2699
v_dot_in = 1
conc_in = 0
v_dot_out = 1

# Function arguments
t = 0.0
dt = 0.25
xk = cas.vertcat(L, m)
uk = cas.vertcat(v_dot_in, conc_in, v_dot_out)
N = 1000
X = np.full((N+1, 2), np.nan)
for k in range(N):
    X[k, :] = xk.T
    xk = tank_model_dt.F(t, xk, uk)
    t += dt
X[N, :] = xk.T
m_final = A * L * conc_in
print(xk)
assert np.allclose(xk, [[L], [1.6280117097544862]])

In [ ]:
# Test 5 - response with initial mass and zero inlet concentration in tank

# Simulation time
Ts = 0.25
t_eval = Ts * np.arange(nT+1)

# Input sequence
v_dot_in = 1
conc_in = 0
v_dot_out = 1
U = np.full((nT, nu), np.nan)
U[:, 0] = v_dot_in
U[:, 1] = conc_in
U[:, 2] = v_dot_out
assert U.shape == (nT, nu)

# Initial condition
L = 5
m = 39.2699
x0 = [L, m]

assert t_eval.shape == (nT + 1, )
X, Y = simulate(t_eval, U, x0)
assert X.shape == (nT + 1, n)
assert Y.shape == (nT + 1, ny)

print(X[-1, :])
assert np.allclose(X[-1, :], [L, 1.6280117097544862])

In [ ]:
ax = make_tank_tsplot(X, dt, m_final)
plt.show()

In [ ]:
# Test 6 - response with initial mass and zero inlet concentration in tank

# Simulation time
Ts = 0.25
t_eval = Ts * np.arange(nT+1)

# Input sequence
v_dot_in = 1.5
conc_in = 0.1
v_dot_out = 1
U = np.full((nT, nu), np.nan)
U[:, 0] = v_dot_in
U[:, 1] = conc_in
U[:, 2] = v_dot_out
assert U.shape == (nT, nu)

# Initial condition
L = 5
m = 25
x0 = [L, m]

assert t_eval.shape == (nT + 1, )
X, Y = simulate(t_eval, U, x0)
assert X.shape == (nT + 1, n)
assert Y.shape == (nT + 1, ny)

print(X[-1, :])
assert np.allclose(X[-1, :], [12.957747154594962, 22.90694328238339])

In [ ]:
ax = make_tank_tsplot(X, dt)
plt.show()

In [ ]:
# Test 7 - Step inputs

# Simulation time
Ts = 0.25
t = Ts * np.arange(nT)

# Input sequence
v_dot_in = [1, 5]
conc_in = [0.1, 0.2]
v_dot_out = [1, 1.5]
U = np.full((nT, nu), [v_dot_in[0], conc_in[0], v_dot_out[0]])
U[(t >= 25) & (t < 50), 0] = v_dot_in[1]
U[(t >= 25) & (t < 50), 1] = conc_in[0]
U[(t >= 75) & (t < 100), 2] = v_dot_in[1]
U[(t >= 75) & (t < 100), 1] = conc_in[1]
U[(t >= 100) & (t < 150), 2] = v_dot_out[1]
assert U.shape == (nT, nu)

# Initial condition
L = 8
m = 12.5663
x0 = [L, m]

assert t_eval.shape == (nT + 1, )
X, Y = simulate(t_eval, U, x0)
assert X.shape == (nT + 1, n)
assert Y.shape == (nT + 1, ny)

ax = make_tank_tsplot(X, dt)
plt.show()

## Analytical Solution

In [ ]:
def ivp_solution(t, x1_init, x2_init, u1, u2, u3, A, exp=np.exp, log=np.log):
    return [
        x1_init + t * (u1 - u3) / A,
        u2 * (A * x1_init + t * u1 - t * u3)
        + (
            (A * x1_init) ** (u3 / (u1 - u3)) * x2_init
            - u2 * exp(u1 * log(A * x1_init) / (u1 - u3))
        ) * exp(-u3 * log(A * x1_init + t * u1 - t * u3) / (u1 - u3))
    ]

# Test with same parameters as in Example 6 above
D = 5  # tank diameter [m]
A = np.pi * D
L = 5
m = 25
v_dot_in = 1.5
conc_in = 0.1
v_dot_out = 1
assert ivp_solution(0.25, L, m, v_dot_in, conc_in, v_dot_out, A) \
    == [5.007957747154594, 24.958052546626035]
v_dot_in = v_dot_out

In [ ]:
# Compare analytical solution to RK4 numerical solution
D = 5  # tank diameter [m]
A = np.pi * D
rng = np.random.default_rng(0)
for _ in range(10):
    L = float(rng.uniform(1, 5))
    m = float(rng.uniform(10, 30))
    v_dot_in = float(rng.uniform(0, 2))
    conc_in = float(rng.uniform(0, 1))
    v_dot_out = float(rng.uniform(0, 2))
    xkp1_anal = ivp_solution(dt, L, m, v_dot_in, conc_in, v_dot_out, A)
    xk = cas.vertcat(L, m)
    uk = cas.vertcat(v_dot_in, conc_in, v_dot_out)
    xkp1_num = np.array(tank_model_dt.F(0.0, xk, uk)).reshape(-1)
    assert np.allclose(xkp1_anal, xkp1_num)

In [ ]:
# Test 6 - response with initial mass and zero inlet concentration in tank

# Simulation time
Ts = 0.25
t_eval = Ts * np.arange(nT+1)

# Input sequence
v_dot_in = 1.5
conc_in = 0.1
v_dot_out = 1
U = np.full((nT, nu), np.nan)
U[:, 0] = v_dot_in
U[:, 1] = conc_in
U[:, 2] = v_dot_out
assert U.shape == (nT, nu)

# Initial condition
L = 5
m = 25
x0 = [L, m]

assert t_eval.shape == (nT + 1, )
X, Y = simulate(t_eval, U, x0)
assert X.shape == (nT + 1, n)
assert Y.shape == (nT + 1, ny)

assert np.allclose(X[-1, :], [12.957747154594962, 22.90694328238339])

# Make sure final values are identical
xkp1_anal = ivp_solution(t_eval[-1], L, m, v_dot_in, conc_in, v_dot_out, A)
assert np.allclose(xkp1_anal, X[-1, :])

## Discrete-Time Model Using Analytical Solution

In [ ]:
# Constants
D = 5  # tank diameter [m]
A = float(np.pi * D)

# Symbolic variables
t = cas.SX.sym("t")
L = cas.SX.sym("L")
m = cas.SX.sym("m")
v_dot_in = cas.SX.sym("v_dot_in")
conc_in = cas.SX.sym("conc_in")
v_dot_out = cas.SX.sym("v_dot_out")

dt = 1
xk = cas.vertcat(L, m)
uk = cas.vertcat(v_dot_in, conc_in, v_dot_out)
xkp1 = cas.vcat(
    ivp_solution(
        dt, L, m, v_dot_in, conc_in, v_dot_out, A, exp=cas.exp, log=cas.log
    )
)

# State transition function
F = cas.Function(
    "F",
    [t, xk, uk],
    [xkp1],
    ["t", "xk", "uk"],
    ["xkp1"],
)

# Output function
yk = xk
H = cas.Function(
    "H",
    [t, xk, uk],
    [yk],
    ["t", "xk", "uk"],
    ["yk"],
)

n = xk.shape[0]
nu = uk.shape[0]
ny = yk.shape[0]

tank_model_dt_anal = StateSpaceModelDT(
    F, 
    H, 
    n, 
    nu, 
    ny, 
    dt=dt,
    input_names=["v_dot_in", "conc_in", "v_dot_out"],
    state_names=["L", "m"],
    output_names=["L", "m"],
)
tank_model_dt_anal